In [18]:
# Example 13-2. A change in the arithmetic context precision may cause x to differ from +x.
import decimal
ctx = decimal.getcontext()
ctx.prec = 40 # ??? decimal.getcontext().prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
print(" one_third = ", one_third)
print("+one_third = ", +one_third)
print("one_third == +one_third : ", one_third == +one_third)
print()
ctx.prec = 28
print(" one_third = ", one_third)
print("+one_third = ", +one_third)
print("one_third == +one_third : ", one_third == +one_third)


 one_third =  0.3333333333333333333333333333333333333333
+one_third =  0.3333333333333333333333333333333333333333
one_third == +one_third :  True

 one_third =  0.3333333333333333333333333333333333333333
+one_third =  0.3333333333333333333333333333
one_third == +one_third :  False


In [29]:
# Example 13-3. Unary + produces a new Counter without zeroed or negative tallies.
from collections import Counter
ct = Counter('abracadabra')
print(" ct = ", ct)
print()
ct['r'] = -3
ct['d'] = 0
print(" ct = ", ct)
print("+ct = ", +ct)

 ct =  Counter({'a': 5, 'r': 2, 'b': 2, 'd': 1, 'c': 1})

 ct =  Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})
+ct =  Counter({'a': 5, 'b': 2, 'c': 1})


In [39]:
# Example 13-4. Vector.__add__ method, take #1
import itertools
pairs = itertools.zip_longest([1,2], [3,4,5], fillvalue=0) #
for x in pairs:
    print(x)
pairs = itertools.zip_longest([1,2], [3,4,5], fillvalue=0) #
for a, b in pairs:
    print(a, b)


(1, 3)
(2, 4)
(0, 5)
1 3
2 4
0 5


In [59]:
# Example 13-11. vector_v7.py: operator * methods added
"""
A multi-dimensional ``Vector`` class, take 7: operator ``*``
A ``Vector`` is built from an iterable of numbers::
    >>> Vector([3.1, 4.2])
    Vector([3.1, 4.2])
    >>> Vector((3, 4, 5))
    Vector([3.0, 4.0, 5.0])
    >>> Vector(range(10))
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
Tests with 2-dimensions (same results as ``vector2d_v1.py``)::
    >>> v1 = Vector([3, 4])
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector([3.0, 4.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector([0, 0]))
    (True, False)
Test of ``.frombytes()`` class method:
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0])
    >>> v1 == v1_clone
    True
Tests with 3-dimensions::
    >>> v1 = Vector([3, 4, 5])
    >>> x, y, z = v1
    >>> x, y, z
    (3.0, 4.0, 5.0)
    >>> v1
    Vector([3.0, 4.0, 5.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0, 5.0)
    >>> abs(v1)  # doctest:+ELLIPSIS
    7.071067811...
    >>> bool(v1), bool(Vector([0, 0, 0]))
    (True, False)
Tests with many dimensions::
    >>> v7 = Vector(range(7))
    >>> v7
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
    >>> abs(v7)  # doctest:+ELLIPSIS
    9.53939201...
Test of ``.__bytes__`` and ``.frombytes()`` methods::
    >>> v1 = Vector([3, 4, 5])
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0, 5.0])
    >>> v1 == v1_clone
    True
Tests of sequence behavior::
    >>> v1 = Vector([3, 4, 5])
    >>> len(v1)
    3
    >>> v1[0], v1[len(v1)-1], v1[-1]
    (3.0, 5.0, 5.0)
Test of slicing::
    >>> v7 = Vector(range(7))
    >>> v7[-1]
    6.0
    >>> v7[1:4]
    Vector([1.0, 2.0, 3.0])
    >>> v7[-1:]
    Vector([6.0])
    >>> v7[1,2]
    Traceback (most recent call last):
      ...
    TypeError: Vector indices must be integers
Tests of dynamic attribute access::
    >>> v7 = Vector(range(10))
    >>> v7.x
    0.0
    >>> v7.y, v7.z, v7.t
    (1.0, 2.0, 3.0)
Dynamic attribute lookup failures::
    >>> v7.k
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'k'
    >>> v3 = Vector(range(3))
    >>> v3.t
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 't'
    >>> v3.spam
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'spam'
Tests of hashing::
    >>> v1 = Vector([3, 4])
    >>> v2 = Vector([3.1, 4.2])
    >>> v3 = Vector([3, 4, 5])
    >>> v6 = Vector(range(6))
    >>> hash(v1), hash(v3), hash(v6)
    (7, 2, 1)
Most hash values of non-integers vary from a 32-bit to 64-bit Python build::
    >>> import sys
    >>> hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)
    True
Tests of ``format()`` with Cartesian coordinates in 2D::
    >>> v1 = Vector([3, 4])
    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'
Tests of ``format()`` with Cartesian coordinates in 3D and 7D::
    >>> v3 = Vector([3, 4, 5])
    >>> format(v3)
    '(3.0, 4.0, 5.0)'
    >>> format(Vector(range(7)))
    '(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'
Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D::
    >>> format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector([1, 1]), '.3eh')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector([1, 1]), '0.5fh')
    '<1.41421, 0.78540>'
    >>> format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.73205..., 0.95531..., 0.78539...>'
    >>> format(Vector([2, 2, 2]), '.3eh')
    '<3.464e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 0, 0]), '0.5fh')
    '<0.00000, 0.00000, 0.00000>'
    >>> format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS
    '<2.0, 2.09439..., 2.18627..., 3.92699...>'
    >>> format(Vector([2, 2, 2, 2]), '.3eh')
    '<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 1, 0, 0]), '0.5fh')
    '<1.00000, 1.57080, 0.00000, 0.00000>'
Unary operator tests::
    >>> v1 = Vector([3, 4])
    >>> abs(v1)
    5.0
    >>> -v1
    Vector([-3.0, -4.0])
    >>> +v1
    Vector([3.0, 4.0])
Basic tests of operator ``+``::
    >>> v1 = Vector([3, 4, 5])
    >>> v2 = Vector([6, 7, 8])
    >>> v1 + v2
    Vector([9.0, 11.0, 13.0])
    >>> v1 + v2 == Vector([3+6, 4+7, 5+8])
    True
    >>> v3 = Vector([1, 2])
    >>> v1 + v3  # short vectors are filled with 0.0 on addition
    Vector([4.0, 6.0, 5.0])
Tests of ``+`` with mixed types::
    >>> v1 + (10, 20, 30)
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v1 + v2d
    Vector([4.0, 6.0, 5.0])
Tests of ``+`` with mixed types, swapped operands::
    >>> (10, 20, 30) + v1
    Vector([13.0, 24.0, 35.0])
    >>> from vector2d_v3 import Vector2d
    >>> v2d = Vector2d(1, 2)
    >>> v2d + v1
    Vector([4.0, 6.0, 5.0])
Tests of ``+`` with an unsuitable operand:
    >>> v1 + 1
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'int'
    >>> v1 + 'ABC'
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'Vector' and 'str'
Basic tests of operator ``*``::
    >>> v1 = Vector([1, 2, 3])
    >>> v1 * 10
    Vector([10.0, 20.0, 30.0])
    >>> 10 * v1
    Vector([10.0, 20.0, 30.0])
Tests of ``*`` with unusual but valid operands::
    >>> v1 * True
    Vector([1.0, 2.0, 3.0])
    >>> from fractions import Fraction
    >>> v1 * Fraction(1, 3)  # doctest:+ELLIPSIS
    Vector([0.3333..., 0.6666..., 1.0])
Tests of ``*`` with unsuitable operands::
    >>> v1 * (1, 2)
    Traceback (most recent call last):
      ...
    TypeError: can't multiply sequence by non-int of type 'Vector'
"""
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components)
    def __iter__(self):
        return iter(self._components)
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))
    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    def __neg__(self):
        return Vector(-x for x in self)
    def __pos__(self):
        return Vector(self)
    def __bool__(self):
        return bool(abs(self))
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    def __rmul__(self, scalar):
        return self * scalar
    
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented
    def __rmatmul__(self, other):
        return self @ other
    
#    def __eq__(self, other):
#        return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented

v1 = Vector([1.0, 2.0, 3.0])
print("14 * v1 = ", 14 * v1)
print("v1 * True = ", v1 * True)

from fractions import Fraction
print("v1 * Fraction(1, 3) = ", v1 * Fraction(1, 3))
print()

v2 = Vector([1, 2, 3])
v3 = Vector([4, 5, 6])
print("v2 @ v3 : ", v2 @ v3)
print()

va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
print("va == vb : ", va == vb)

import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')
from vector2d_v3 import Vector2d
vc = Vector([1, 2])
v2d = Vector2d(1, 2)
print("vc == v2d : ", vc == v2d)

t3 = (1, 2, 3)
print("va == t3 : ", va == t3)
print()

print("va != vb : ", va != vb)
print("vc != v2d : ", vc != v2d)
print("va != t3 : ", va != t3)

14 * v1 =  (14.0, 28.0, 42.0)
v1 * True =  (1.0, 2.0, 3.0)
v1 * Fraction(1, 3) =  (0.3333333333333333, 0.6666666666666666, 1.0)

v2 @ v3 :  32.0

va == vb :  True
vc == v2d :  True
va == t3 :  False

va != vb :  False
vc != v2d :  False
va != t3 :  True


In [65]:
# Example 13-15. Augmented assignment works with immutable targets by creating new instances and rebinding.
v1 = Vector([1, 2, 3])
v1_alias = v1 #
print("id(v1) : ", id(v1))

v1 += Vector([4, 5, 6]) #
print("v1 += Vector([4, 5, 6]) : ", v1)
print("id(v1) : ", id(v1))

v1 *= 11
print("v1 *= 11 : ", v1)
print("id(v1) : ", id(v1))

print("v1_alias : ", v1_alias)

id(v1) :  139750213307192
v1 += Vector([4, 5, 6]) :  (5.0, 7.0, 9.0)
id(v1) :  139750213306968
v1 *= 11 :  (55.0, 77.0, 99.0)
id(v1) :  139750212974408
v1_alias :  (1.0, 2.0, 3.0)


In [67]:
# Example 13-16. A new AddableBingoCage instance can be created with +.
"""
======================
AddableBingoCage tests
======================
Tests for __add__:
# BEGIN ADDABLE_BINGO_ADD_DEMO
    >>> vowels = 'AEIOU'
    >>> globe = AddableBingoCage(vowels)  # <1>
    >>> globe.inspect()
    ('A', 'E', 'I', 'O', 'U')
    >>> globe.pick() in vowels  # <2>
    True
    >>> len(globe.inspect())  # <3>
    4
    >>> globe2 = AddableBingoCage('XYZ')  # <4>
    >>> globe3 = globe + globe2
    >>> len(globe3.inspect())  # <5>
    7
    >>> void = globe + [10, 20]  # <6>
    Traceback (most recent call last):
      ...
    TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'
# END ADDABLE_BINGO_ADD_DEMO
Tests for __iadd__:
# BEGIN ADDABLE_BINGO_IADD_DEMO
    >>> globe_orig = globe  # <1>
    >>> len(globe.inspect())  # <2>
    4
    >>> globe += globe2  # <3>
    >>> len(globe.inspect())
    7
    >>> globe += ['M', 'N']  # <4>
    >>> len(globe.inspect())
    9
    >>> globe is globe_orig  # <5>
    True
    >>> globe += 1  # <6>
    Traceback (most recent call last):
      ...
    TypeError: right operand in += must be 'AddableBingoCage' or an iterable
# END ADDABLE_BINGO_IADD_DEMO
"""
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

# BEGIN ADDABLE_BINGO
import itertools  # <1>

from tombola import Tombola
from bingo import BingoCage


class AddableBingoCage(BingoCage):  # <2>

    def __add__(self, other):
        if isinstance(other, Tombola):  # <3>
            return AddableBingoCage(self.inspect() + other.inspect())  # <6>
        else:
            return NotImplemented

    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()  # <4>
        else:
            try:
                other_iterable = iter(other)  # <5>
            except TypeError:  # <6>
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)  # <7>
        return self  # <8>

# END ADDABLE_BINGO

